# Search algorithms within Optuna

In this notebook, I will demo how to select the search algorithm with Optuna. We will compare the use of:

* Grid Search
* Randomized search
* Tree-structured Parzen Estimators
* CMA-ES
We can select the search algorithm from the optuna.study.create_study() class.

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.datasets import load_breast_cancer
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.ensemble import RandomForestClassifier

import optuna

In [7]:
# load dataset

breast_cancer_X, breast_cancer_y = load_breast_cancer(return_X_y=True)
X = pd.DataFrame(breast_cancer_X)
y = pd.Series(breast_cancer_y).map({0:1, 1:0})

X.head()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [8]:
# the target:
# percentage of benign (0) and malign tumors (1)

y.value_counts() / len(y)

0    0.627417
1    0.372583
dtype: float64

In [9]:
# split dataset into a train and test set

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=0)

X_train.shape, X_test.shape

((398, 30), (171, 30))

### Define the objective function

This is the hyperparameter response space, the function we want to minimize.

In [4]:
# the objective function takes the hyperparameter space
# as input

def objective(trial):

    rf_n_estimators = trial.suggest_int("rf_n_estimators", 100, 1000)
    rf_criterion = trial.suggest_categorical("rf_criterion", ['gini', 'entropy'])
    rf_max_depth = trial.suggest_int("rf_max_depth", 1, 4)
    rf_min_samples_split = trial.suggest_float("rf_min_samples_split", 0.01, 1)
    
    model = RandomForestClassifier(
        n_estimators=rf_n_estimators,
        criterion=rf_criterion,
        max_depth=rf_max_depth,
        min_samples_split=rf_min_samples_split,
    )

    score = cross_val_score(model, X_train, y_train, cv=3)
    accuracy = score.mean()
    return accuracy

### Randomized Search

RandomSampler()

In [10]:
study = optuna.create_study(
    direction="maximize",
    sampler=optuna.samplers.RandomSampler(),
)


study.optimize(objective, n_trials=5)

[I 2022-10-27 05:31:53,003] A new study created in memory with name: no-name-00089572-5729-4271-904e-bbefc829b3fb
[I 2022-10-27 05:31:54,658] Trial 0 finished with value: 0.6256360598465861 and parameters: {'rf_n_estimators': 442, 'rf_criterion': 'entropy', 'rf_max_depth': 2, 'rf_min_samples_split': 0.7616728064347049}. Best is trial 0 with value: 0.6256360598465861.
[I 2022-10-27 05:31:58,182] Trial 1 finished with value: 0.6256360598465861 and parameters: {'rf_n_estimators': 866, 'rf_criterion': 'gini', 'rf_max_depth': 3, 'rf_min_samples_split': 0.9159691112867344}. Best is trial 0 with value: 0.6256360598465861.
[I 2022-10-27 05:31:59,033] Trial 2 finished with value: 0.9372294372294373 and parameters: {'rf_n_estimators': 203, 'rf_criterion': 'gini', 'rf_max_depth': 2, 'rf_min_samples_split': 0.03632489661992204}. Best is trial 2 with value: 0.9372294372294373.
[I 2022-10-27 05:31:59,534] Trial 3 finished with value: 0.912128806865649 and parameters: {'rf_n_estimators': 109, 'rf_cri

In [11]:
study.best_params

{'rf_n_estimators': 430,
 'rf_criterion': 'gini',
 'rf_max_depth': 4,
 'rf_min_samples_split': 0.08949167120542477}

In [12]:
study.best_value

0.942241968557758

In [13]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_rf_criterion,params_rf_max_depth,params_rf_min_samples_split,params_rf_n_estimators,state
0,0,0.625636,2022-10-27 05:31:53.005684,2022-10-27 05:31:54.658504,0 days 00:00:01.652820,entropy,2,0.761673,442,COMPLETE
1,1,0.625636,2022-10-27 05:31:54.659810,2022-10-27 05:31:58.182451,0 days 00:00:03.522641,gini,3,0.915969,866,COMPLETE
2,2,0.937229,2022-10-27 05:31:58.183828,2022-10-27 05:31:59.033397,0 days 00:00:00.849569,gini,2,0.036325,203,COMPLETE
3,3,0.912129,2022-10-27 05:31:59.034616,2022-10-27 05:31:59.534231,0 days 00:00:00.499615,gini,1,0.420107,109,COMPLETE
4,4,0.942242,2022-10-27 05:31:59.536029,2022-10-27 05:32:03.466756,0 days 00:00:03.930727,gini,4,0.089492,430,COMPLETE


### TPE

TPESampler is the default

In [14]:
study = optuna.create_study(
    direction="maximize",
    sampler=optuna.samplers.TPESampler(),
)


study.optimize(objective, n_trials=5)

[I 2022-10-27 05:32:28,081] A new study created in memory with name: no-name-3aeb48b8-dde5-4efc-98f9-b9f89ab2d117
[I 2022-10-27 05:32:30,613] Trial 0 finished with value: 0.6256360598465861 and parameters: {'rf_n_estimators': 667, 'rf_criterion': 'entropy', 'rf_max_depth': 3, 'rf_min_samples_split': 0.9763441693740854}. Best is trial 0 with value: 0.6256360598465861.
[I 2022-10-27 05:32:31,136] Trial 1 finished with value: 0.6256360598465861 and parameters: {'rf_n_estimators': 131, 'rf_criterion': 'gini', 'rf_max_depth': 4, 'rf_min_samples_split': 0.7301136212165581}. Best is trial 0 with value: 0.6256360598465861.
[I 2022-10-27 05:32:34,899] Trial 2 finished with value: 0.9396787423103211 and parameters: {'rf_n_estimators': 818, 'rf_criterion': 'entropy', 'rf_max_depth': 4, 'rf_min_samples_split': 0.3763767509612783}. Best is trial 2 with value: 0.9396787423103211.
[I 2022-10-27 05:32:35,315] Trial 3 finished with value: 0.9146160856687172 and parameters: {'rf_n_estimators': 101, 'rf_

In [15]:
study.best_params

{'rf_n_estimators': 818,
 'rf_criterion': 'entropy',
 'rf_max_depth': 4,
 'rf_min_samples_split': 0.3763767509612783}

In [16]:
study.best_value

0.9396787423103211

### CMA-ES

CmaEsSampler

In [17]:
study = optuna.create_study(
    direction="maximize",
    sampler=optuna.samplers.CmaEsSampler(),
)

study.optimize(objective, n_trials=5)

[I 2022-10-27 05:32:59,479] A new study created in memory with name: no-name-d951b97b-f130-4972-addd-1a83b35920c7
[I 2022-10-27 05:33:01,270] Trial 0 finished with value: 0.9171413381939697 and parameters: {'rf_n_estimators': 431, 'rf_criterion': 'entropy', 'rf_max_depth': 3, 'rf_min_samples_split': 0.6267530272470063}. Best is trial 0 with value: 0.9171413381939697.
[W 2022-10-27 05:33:01,329] The parameter 'rf_criterion' in trial#1 is sampled independently by using `RandomSampler` instead of `CmaEsSampler` (optimization performance may be degraded). `CmaEsSampler` does not support dynamic search space or `CategoricalDistribution`. You can suppress this warning by setting `warn_independent_sampling` to `False` in the constructor of `CmaEsSampler`, if this independent sampling is intended behavior.
[I 2022-10-27 05:33:03,690] Trial 1 finished with value: 0.9396977291714134 and parameters: {'rf_n_estimators': 550, 'rf_criterion': 'gini', 'rf_max_depth': 2, 'rf_min_samples_split': 0.2592

In [18]:
study.best_params

{'rf_n_estimators': 550,
 'rf_criterion': 'gini',
 'rf_max_depth': 2,
 'rf_min_samples_split': 0.15643475222342784}

In [19]:
study.best_value

0.9397357028935976

### Grid Search

GridSampler()

We are probably not going to perform GridSearch with Optuna, but in case you wanted to, you need to add a variable with the space, with the exact values that you want to be tested.

In [20]:
search_space = {
    "rf_n_estimators": [100, 500, 1000],
    "rf_criterion": ['gini', 'entropy'],
    "rf_max_depth": [1, 2, 3],
    "rf_min_samples_split": [0.1, 1.0]
}

In [21]:
study = optuna.create_study(
    direction="maximize",
    sampler=optuna.samplers.GridSampler(search_space),
)

study.optimize(objective)

[I 2022-10-27 05:33:46,610] A new study created in memory with name: no-name-c3a03bf8-53b7-4786-9c2c-07ab5ca088e5
[I 2022-10-27 05:33:47,072] Trial 0 finished with value: 0.6256360598465861 and parameters: {'rf_n_estimators': 100, 'rf_criterion': 'entropy', 'rf_max_depth': 3, 'rf_min_samples_split': 1.0}. Best is trial 0 with value: 0.6256360598465861.
[I 2022-10-27 05:33:51,606] Trial 1 finished with value: 0.9397357028935976 and parameters: {'rf_n_estimators': 1000, 'rf_criterion': 'gini', 'rf_max_depth': 3, 'rf_min_samples_split': 0.1}. Best is trial 1 with value: 0.9397357028935976.
[I 2022-10-27 05:33:52,191] Trial 2 finished with value: 0.942222981696666 and parameters: {'rf_n_estimators': 100, 'rf_criterion': 'gini', 'rf_max_depth': 3, 'rf_min_samples_split': 0.1}. Best is trial 2 with value: 0.942222981696666.
[I 2022-10-27 05:33:53,933] Trial 3 finished with value: 0.6256360598465861 and parameters: {'rf_n_estimators': 500, 'rf_criterion': 'entropy', 'rf_max_depth': 2, 'rf_min

In [22]:
study.best_params

{'rf_n_estimators': 500,
 'rf_criterion': 'gini',
 'rf_max_depth': 3,
 'rf_min_samples_split': 0.1}

In [23]:
study.best_value

0.942241968557758